In [1]:
class Board(dict):
    def __init__(self):
        super().__init__()
        for key in 'abcdefghi':
            self[key] = None
        self.count = 0
        self.winning_combinations = [
            'abc', 'def', 'ghi',  # Filas
            'adg', 'beh', 'cfi',  # Columnas
            'aei', 'ceg'          # Diagonales
        ]

    def __str__(self):
        """Return a string representation of the board."""
        return (
            f"{self['a']} | {self['b']} | {self['c']}\n"
            f"---------\n"
            f"{self['d']} | {self['e']} | {self['f']}\n"
            f"---------\n"
            f"{self['g']} | {self['h']} | {self['i']}"
        ).replace('None', ' ')

    def reset(self):
        """Regresa el caso del tablero al estado inicial."""
        for key in self:
            self[key] = None
        self.count = 0

    def set_value(self, key, value):
        if self.checkState() == None:
            """Cambia el valor de una de las llaves."""
            if key not in self:
                raise KeyError(f"Llave Invalida: {key}")
            self[key] = value
            self.count = self.count + 1
            if self.count >= 3:
                self.checkState()
    
    def is_won(self):
        """Ve las posibles combinaciones con las que gana algún jugador. Regresa si el juego fue ganado y por quien"""
        for combo in self.winning_combinations:
            values = [self[key] for key in combo]
            if values[0] is not None and all(v == values[0] for v in values):
                return values[0]
        return False

    def is_tied(self):
        """Regresa si el juego es un empate."""
        return self.count == 9 and not self.is_won()

    def checkState(self):
        """Revisa si el juego esta terminado. Y regresa como quedo"""
        winner = self.is_won()
        if winner != False:
            return winner
        elif self.is_tied():
            return "Empatado"
        else:
            return None
        

In [2]:
# Usage example:
board = Board()
print("Initial board:")
print(board)
print("\nSetting some values:")
board.set_value('a', 'X')
board.set_value('b', 'O')
board.set_value('e', 'X')
board.set_value('h', 'O')
board.set_value('f', 'X')
board.set_value('c', 'O')
board.set_value('g', 'X')
board.set_value('d', 'O')
board.set_value('i', 'O')


print(board)
print(board.count)
print(board.checkState())


Initial board:
  |   |  
---------
  |   |  
---------
  |   |  

Setting some values:
X | O | O
---------
O | X | X
---------
X | O | O
9
Empatado


In [4]:
class Meta_Board(dict):
    def __init__(self):
        super().__init__()
        for key in 'ABCDEFGHI':
            self[key] = Board()
        self.count = 0
        self.winning_combinations = [
            'ABC', 'DEF', 'GHI',  # Filas
            'ADG', 'BEH', 'CFI',  # Columnas
            'AEI', 'CEG'          # Diagonales
        ]
        self.current_board = None

    def __str__(self):
        """Devuelve una representación en cadena del Meta_Board."""
        meta_rows = ['ABC', 'DEF', 'GHI']
        rows = ['abc', 'def', 'ghi']
        meta_str = ""
        
        for meta_row in meta_rows:
            # Dividir cada tablero en sus tres filas
            board_rows = [str(self[key]).split('\n') for key in meta_row]
            # Combinar las filas de los tres tableros en cada fila del meta-tablero
            for i in range(5):
                meta_str += " | ".join(board_rows[j][i] for j in range(3)) + "\n"
            
            if meta_row != meta_rows[-1]:
                meta_str += "-" * 34 + "\n"
        
        return meta_str

    def reset(self):
        """Regresa el caso del Meta_Board al estado inicial."""
        for key in self:
            self[key].reset()
        self.count = 0
        self.current_board = None

    def set_value(self, meta_key, board_key, value):
        """Cambia el valor de una de las llaves en un Board específico."""
        if self.checkState() is None:
            if meta_key not in self:
                raise KeyError(f"Llave de Meta_Board Inválida: {meta_key}")
            if self.current_board is None or self.current_board == meta_key:
                self[meta_key].set_value(board_key, value)
                self.count = self.count + 1
                if self[meta_key].checkState() is not None:
                    self.current_board = None
                else:
                    self.current_board = board_key.upper()
            else:
                raise ValueError(f"Movimiento inválido. Debe jugar en el tablero {self.current_board}")

    def is_won(self):
        """Ve las posibles combinaciones con las que gana algún jugador en el Meta_Board."""
        for combo in self.winning_combinations:
            values = [self[key].checkState() for key in combo]
            if values[0] is not None and values[0] != "Empatado" and all(v == values[0] for v in values):
                return values[0]
        return False

    def is_tied(self):
        """Regresa si el Meta_Board es un empate."""
        return self.count == 36 and not self.is_won()

    def checkState(self):
        """Revisa si el juego en el Meta_Board está terminado y regresa cómo quedó."""
        winner = self.is_won()
        if winner:
            return winner
        elif self.is_tied():
            return "Empatado"
        else:
            return None

# Ejemplo de uso:
meta_board = Meta_Board()
print("Meta_Board inicial:")
print(meta_board)

print("\nHaciendo algunos movimientos:")
meta_board.set_value('A', 'a', 'X')
meta_board.set_value('A', 'h', 'O')
meta_board.set_value('H', 'e', 'X')
print(meta_board)

print(f"\nEstado del juego: {meta_board.checkState()}")
print(f"Tablero actual: {meta_board.current_board}")

Meta_Board inicial:
  |   |   |   |   |   |   |   |  
--------- | --------- | ---------
  |   |   |   |   |   |   |   |  
--------- | --------- | ---------
  |   |   |   |   |   |   |   |  
----------------------------------
  |   |   |   |   |   |   |   |  
--------- | --------- | ---------
  |   |   |   |   |   |   |   |  
--------- | --------- | ---------
  |   |   |   |   |   |   |   |  
----------------------------------
  |   |   |   |   |   |   |   |  
--------- | --------- | ---------
  |   |   |   |   |   |   |   |  
--------- | --------- | ---------
  |   |   |   |   |   |   |   |  


Haciendo algunos movimientos:
X |   |   |   |   |   |   |   |  
--------- | --------- | ---------
  |   |   |   |   |   |   |   |  
--------- | --------- | ---------
  | O |   |   |   |   |   |   |  
----------------------------------
  |   |   |   |   |   |   |   |  
--------- | --------- | ---------
  |   |   |   |   |   |   |   |  
--------- | --------- | ---------
  |   |   |   |   |   |   |